# Geotiny Seismometer - Quick Start

This notebook provides a quick introduction to working with Geotiny seismometer data.

## Setup

Make sure you have installed all required packages:
```bash
pip install -r requirements.txt
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Optional: uncomment if you have ObsPy installed
# from obspy import read

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

## 1. Load Configuration

In [ ]:
import yaml

with open('../config/geotiny.yml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded:")
print(f"Device IP: {config['device']['ip_address']}")
print(f"Sampling Rate: {config['acquisition']['sampling_rate']} Hz")
print(f"Data Path: {config['storage']['raw_data_path']}")

## 2. Load Sample Data

Load a CSV file from processed data:

In [ ]:
# List available data files
data_path = Path(config['storage']['processed_data_path'])
csv_files = list(data_path.glob('*.csv'))

if csv_files:
    print(f"Found {len(csv_files)} CSV files:")
    for f in csv_files[:5]:  # Show first 5
        print(f"  - {f.name}")
else:
    print("No CSV files found. Run convert_to_csv.py first.")

In [ ]:
# Load the first file (or specify your own)
if csv_files:
    df = pd.read_csv(csv_files[0], parse_dates=['timestamp'])
    print(f"Loaded {len(df)} samples")
    print(f"Channels: {[col for col in df.columns if col != 'timestamp']}")
    print(f"Duration: {df['timestamp'].max() - df['timestamp'].min()}")
    df.head()

## 3. Visualize Time Series

In [ ]:
if csv_files and 'df' in locals():
    channels = [col for col in df.columns if col != 'timestamp']
    
    fig, axes = plt.subplots(len(channels), 1, figsize=(14, 3*len(channels)))
    if len(channels) == 1:
        axes = [axes]
    
    for ax, channel in zip(axes, channels):
        ax.plot(df['timestamp'], df[channel], linewidth=0.7)
        ax.set_ylabel(f'{channel}\n(counts)', fontsize=11)
        ax.grid(True, alpha=0.3)
    
    axes[-1].set_xlabel('Time', fontsize=11)
    fig.suptitle('Seismic Data - Time Series', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 4. Basic Statistics

In [ ]:
if csv_files and 'df' in locals():
    print("Statistical Summary:")
    print(df.describe())

## 5. Frequency Analysis (FFT)

Compute and plot the frequency spectrum:

In [ ]:
if csv_files and 'df' in locals():
    # Compute sampling rate from timestamps
    dt = (df['timestamp'].iloc[1] - df['timestamp'].iloc[0]).total_seconds()
    fs = 1 / dt
    
    channels = [col for col in df.columns if col != 'timestamp']
    
    fig, axes = plt.subplots(1, len(channels), figsize=(6*len(channels), 4))
    if len(channels) == 1:
        axes = [axes]
    
    for ax, channel in zip(axes, channels):
        # Compute FFT
        signal = df[channel].values
        fft = np.fft.rfft(signal)
        freqs = np.fft.rfftfreq(len(signal), dt)
        
        # Plot power spectrum
        power = np.abs(fft) ** 2
        ax.semilogy(freqs, power)
        ax.set_xlabel('Frequency (Hz)')
        ax.set_ylabel('Power')
        ax.set_title(f'{channel} - Frequency Spectrum')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## Next Steps

- Apply filters (bandpass, highpass, lowpass)
- Detect events/triggers
- Compare multiple recordings
- Generate spectrograms
- Export processed data